In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
import os
from matplotlib import pyplot as plt
import seaborn as sbn

In [3]:
%matplotlib inline
plt.style.use('seaborn-paper')

In [4]:
os.listdir('..')

['.DS_Store',
 'links (table 3).csv',
 'routes (table 4).csv',
 'trajectories(table 5)_training.csv',
 'volume(table 6)_training.csv',
 'weather (table 7)_training.csv',
 'scripts',
 '__MACOSX',
 'training_20min_avg_travel_time.csv',
 'training_20min_avg_volume.csv',
 'A2.csv',
 'A3.csv',
 'B1.csv',
 'B3.csv',
 'C1.csv',
 'C3.csv',
 'test1_20min_avg_volume.csv',
 'test1_20min_avg_travel_time.csv']

In [5]:
data = {
    'links': pd.read_csv('../links (table 3).csv'),
    'routes': pd.read_csv('../routes (table 4).csv'),
    'trajectories': pd.read_csv('../trajectories(table 5)_training.csv'),
    'volume': pd.read_csv('../volume(table 6)_training.csv'),
    'weather': pd.read_csv('../weather (table 7)_training.csv'),
    'avg_travel_time': pd.read_csv('../training_20min_avg_travel_time.csv'),
    'avg_volume': pd.read_csv('../training_20min_avg_volume.csv')
}

links = data['links']
routes = data['routes']
trajectories = data['trajectories']
volume = data['volume']
weather = data['weather']
avg_travel_time = data['avg_travel_time']
avg_volume = data['avg_volume']

In [6]:
from IPython.display import display, HTML

for table_name in data:
    table = data[table_name]
    print(table_name)
    display(table.head(5))

weather


,date,hour,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation
0,2016-07-01,0,1000.4,1005.3,225.0,2.1,26.4,94.0,0.0
1,2016-07-01,3,1000.5,1005.3,187.0,2.7,29.0,76.0,0.0
2,2016-07-01,6,998.9,1003.7,212.0,2.9,31.7,67.0,0.0
3,2016-07-01,9,998.7,1003.5,244.0,2.7,31.6,59.0,0.0
4,2016-07-01,12,999.7,1004.5,222.0,1.3,29.9,68.0,0.0


links


,link_id,length,width,lanes,in_top,out_top,lane_width
0,100,58,3,1,105,111,3
1,101,84,3,1,116,121,3
2,102,131,9,3,115,109,3
3,103,23,12,4,111,"122,116",3
4,104,293,9,3,109,112,3


avg_travel_time


...

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


routes


,intersection_id,tollgate_id,link_seq
0,A,2,"110,123,107,108,120,117"
1,A,3,"110,123,107,108,119,114,118,122"
2,B,1,"105,100,111,103,116,101,121,106,113"
3,B,3,"105,100,111,103,122"
4,C,1,"115,102,109,104,112,111,103,116,101,121,106,113"


trajectories


,intersection_id,tollgate_id,vehicle_id,starting_time,travel_seq,travel_time
0,B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...,70.85
1,B,3,1047198,2016-07-19 00:35:56,105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...,148.79
2,B,1,1086390,2016-07-19 00:37:15,105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...,79.76
3,A,2,1071181,2016-07-19 00:37:59,110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...,58.05
4,B,1,1065807,2016-07-19 00:56:21,105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...,137.98


In [7]:
def split_time_window(time_window):
    time_start = [time[1:-1].split(',')[0] for time in time_window]
    time_end = [time[1:-1].split(',')[1] for time in time_window]

    return pd.to_datetime(pd.Series(time_start)), pd.to_datetime(pd.Series(time_end))

In [8]:
avg_travel_time['time_start'], avg_travel_time['time_end'] = split_time_window(avg_travel_time.time_window)
avg_travel_time = avg_travel_time.drop(['time_window'], axis=1)

avg_volume['time_start'], avg_volume['time_end'] = split_time_window(avg_volume.time_window)
avg_volume = avg_volume.drop(['time_window'], axis=1)

In [9]:
avg_travel_time.head()

,intersection_id,tollgate_id,avg_travel_time,time_start,time_end
0,B,3,70.85,2016-07-19 00:00:00,2016-07-19 00:20:00
1,B,3,148.79,2016-07-19 00:20:00,2016-07-19 00:40:00
2,B,3,93.72,2016-07-19 01:40:00,2016-07-19 02:00:00
3,B,3,67.81,2016-07-19 02:00:00,2016-07-19 02:20:00
4,B,3,167.55,2016-07-19 02:40:00,2016-07-19 03:00:00


In [10]:
avg_travel_time[avg_travel_time.intersection_id == 'A'].head()

,intersection_id,tollgate_id,avg_travel_time,time_start,time_end
4803,A,3,94.57,2016-07-19 01:20:00,2016-07-19 01:40:00
4804,A,3,77.61,2016-07-19 02:20:00,2016-07-19 02:40:00
4805,A,3,93.09,2016-07-19 03:00:00,2016-07-19 03:20:00
4806,A,3,91.61,2016-07-19 04:40:00,2016-07-19 05:00:00
4807,A,3,168.36,2016-07-19 05:20:00,2016-07-19 05:40:00


In [11]:
# Time Seq
def get_avg_time(intersection_id, tollgate_id):
    intersection = avg_travel_time[avg_travel_time.intersection_id == intersection_id]
    table = intersection[intersection.tollgate_id == tollgate_id]
    if len(table) == 0:
        return None
    return table[['time_start','avg_travel_time']].set_index('time_start')

In [12]:
names = ['A', 'B', 'C']
tollgates = [1, 2, 3]

for name in names:
    for tollgate in tollgates:
        seq = get_avg_time(name, tollgate)
        
        if seq is not None:
            seq.to_csv('../{}{}.csv'.format(name, tollgate))

In [13]:
def handleTime(time):
    return pd.DataFrame({
        "date": pd.Series([t.date() for t in time]),
        "day_in_month": pd.Series([t.daysinmonth for t in time]), 
        "day_of_year": pd.Series([t.dayofyear for t in time]),
        "day_of_week": pd.Series([t.dayofweek for t in time]),
        "hour": pd.Series([t.hour for t in time]),
        "month": pd.Series([t.month for t in time]),
        "minute": pd.Series([t.minute for t in time]),
    })

In [14]:
test_avg_volume = pd.read_csv('../test1_20min_avg_travel_time.csv')

test_avg_volume['time_start'], test_avg_volume['time_end'] = split_time_window(test_avg_volume.time_window)
test_avg_volume = test_avg_volume.drop(['time_window'], axis=1)

# 处理Avg Travel Time

In [17]:
from sklearn.preprocessing import LabelEncoder
def avg_travel_time_preprocess(avg_travel_time, weather):
    table = avg_travel_time.join(handleTime(avg_travel_time.time_start))
    '''
    weather.date = pd.to_datetime(weather.date)
    weather.date = [t.date() for t in weather.date]

    table.merge(weather, on=['date','hour'])
    '''
    enconder = LabelEncoder()
    
    table.intersection_id = enconder.fit_transform(y=table.intersection_id)
    return table

In [18]:
table = avg_travel_time_preprocess(avg_travel_time, weather)
test_table = avg_travel_time_preprocess(test_avg_volume, weather)

In [123]:
from sklearn.metrics import make_scorer

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))

loss = make_scorer(mean_absolute_percentage_error)

In [105]:
ind = (table['time_start'] > '2015-7-1') & (table['time_start'] < '2017-8-1')
train_x = table[ind]
train_x = train_x.drop(['avg_travel_time', 'time_start', 'time_end', 'date'], axis=1)
train_y = avg_travel_time.avg_travel_time[ind]

'''
from sklearn.feature_selection import SelectFromModel
selector = SelectFromModel(model)
selector.fit(X=train_x, y=train_y)
train_x = selector.fit_transform(X=train_x, y=train_y)
'''

'\nfrom sklearn.feature_selection import SelectFromModel\nselector = SelectFromModel(model)\nselector.fit(X=train_x, y=train_y)\ntrain_x = selector.fit_transform(X=train_x, y=train_y)\n'

In [124]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.externals import joblib

# model = xgb.XGBRegressor(max_depth=5,
#                          learning_rate=0.1,
#                          n_estimators=400,
#                          silent=False, 
#                          objective='reg:linear', 
#                          gamma=0, 
#                          min_child_weight=1, 
#                          max_delta_step=0, 
#                          subsample=1, 
#                          colsample_bytree=1, 
#                          colsample_bylevel=1, 
#                          reg_alpha=0, 
#                          reg_lambda=1, 
#                          base_score=0.5, 
#                          seed=0, 
#                          missing=None)

model = RandomForestRegressor()

model = model.fit(X=train_x, y=train_y)

parameters = {
    'n_estimators':(10, 20, 50),
    'min_samples_split':(2,3),
}

model = GridSearchCV(model, parameters, scoring=loss, n_jobs=-1)
model.fit(train_x, train_y)

model.best_params_

{'min_samples_split': 2, 'n_estimators': 10}

In [125]:
y_pred = model.predict(train_x)
mean_absolute_percentage_error(train_y, y_pred)

0.1034258227042625

In [23]:
model.fit(t2, train_y)
y_pred = model.predict(selector.transform(train_x))
mean_absolute_percentage_error(train_y, y_pred)

NameError: name 't2' is not defined

In [111]:
from sklearn.externals import joblib
joblib.dump(model, 'model.pkl') 

['model.pkl']

In [116]:
model.cv_results_

{'mean_fit_time': array([ 0.3389349 ,  0.65601699,  2.88328171,  0.30546602,  0.59187984,
         2.5291241 ]),
 'mean_score_time': array([ 0.00948834,  0.01910043,  0.07291818,  0.00884191,  0.01568969,
         0.06556018]),
 'mean_test_score': array([ 0.36879043,  0.37023977,  0.35985518,  0.37424282,  0.36377716,
         0.35553236]),
 'mean_train_score': array([ 0.10472752,  0.1003971 ,  0.09579037,  0.12959896,  0.12456811,
         0.12030286]),
 'param_min_samples_split': masked_array(data = [2 2 2 3 3 3],
              mask = [False False False False False False],
        fill_value = ?),
 'param_n_estimators': masked_array(data = [10 20 100 10 20 100],
              mask = [False False False False False False],
        fill_value = ?),
 'params': ({'min_samples_split': 2, 'n_estimators': 10},
  {'min_samples_split': 2, 'n_estimators': 20},
  {'min_samples_split': 2, 'n_estimators': 100},
  {'min_samples_split': 3, 'n_estimators': 10},
  {'min_samples_split': 3, 'n_estimator

In [129]:
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
StackingRegressor(regressors=[model], meta_regressor=lr)